In [ ]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd
import numpy as np

from preprocessing import preprocess
from walkforward import WFCVGenerator
from config.config_types import AppConfig

from utils.paths import CONFIG_DIR, SP500COPY_PATH, SP500_PATH

In [12]:
df_sp = pd.read_parquet(SP500_PATH)

In [20]:
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")

df = preprocess()

df = df.sort_values(by="var", ascending=True)
df

14:11:11 - Preprocessing - DEBUG - number of nans in var: 4980 (preprocessing.py:179)


,permno,date,ret,close,vol,open,high,low,cfacpr,var,t
145646,76841,2009-09-21,0.0178,50.89,2595867.0,49.82,51.36,50.82,1.085115,-1.185758e-04,2443
237206,13103,2023-06-05,-0.015156,92.27,1758253.0,93.67,93.07,91.995,1.0,-2.011509e-05,5892
124375,11762,2023-06-05,-0.015498,183.59,1769410.0,186.38,185.63,183.2,1.0,-1.059414e-06,5892
246768,77976,2011-02-24,0.000209,47.97,1198300.0,47.96,47.97,47.96,1.0,4.942346e-09,2803
246767,77976,2011-02-23,0.0,47.96,1282300.0,47.97,47.97,47.96,1.0,4.942346e-09,2802
...,...,...,...,...,...,...,...,...,...,...,...
204517,86314,2015-08-24,-0.035901,36.79,4215298.0,35.24,37.62,5.4,1.0,1.883290e+00,3934
113499,92602,2010-05-06,-0.035106,47.0,16460800.0,48.58,48.92,2.0,1.0,5.110107e+00,2600
80684,58640,2002-07-16,0.043166,1.45,41692400.0,1.4,1.54,0.01,0.1,1.268497e+01,634
131015,21792,2010-05-06,-0.034771,13.88,9331100.0,14.39,14.39,0.0135,1.0,2.430109e+01,2600


In [ ]:
df["h/l^2"] = df["high"]/df["low"]
df["c/o^2"] = df["close"]/df["open"]
df

,permno,date,ret,close,vol,open,high,low,cfacpr,var,t,h/l,c/o
145646,76841,2009-09-21,0.0178,50.89,2595867.0,49.82,51.36,50.82,1.085115,-1.185758e-04,2443,1.010626,1.021477
237206,13103,2023-06-05,-0.015156,92.27,1758253.0,93.67,93.07,91.995,1.0,-2.011509e-05,5892,1.011685,0.985054
124375,11762,2023-06-05,-0.015498,183.59,1769410.0,186.38,185.63,183.2,1.0,-1.059414e-06,5892,1.013264,0.985031
246768,77976,2011-02-24,0.000209,47.97,1198300.0,47.96,47.97,47.96,1.0,4.942346e-09,2803,1.000209,1.000209
246767,77976,2011-02-23,0.0,47.96,1282300.0,47.97,47.97,47.96,1.0,4.942346e-09,2802,1.000209,0.999792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204517,86314,2015-08-24,-0.035901,36.79,4215298.0,35.24,37.62,5.4,1.0,1.883290e+00,3934,6.966667,1.043984
113499,92602,2010-05-06,-0.035106,47.0,16460800.0,48.58,48.92,2.0,1.0,5.110107e+00,2600,24.46,0.967476
80684,58640,2002-07-16,0.043166,1.45,41692400.0,1.4,1.54,0.01,0.1,1.268497e+01,634,154.0,1.035714
131015,21792,2010-05-06,-0.034771,13.88,9331100.0,14.39,14.39,0.0135,1.0,2.430109e+01,2600,1065.925926,0.964559


In [4]:
wf = WFCVGenerator(cfg.walkforward)



In [5]:
wf.df

,t,var,permno
0,0,0.001527,10078
1,1,0.001098,10078
2,2,0.004587,10078
3,3,0.001987,10078
4,4,0.001109,10078
...,...,...,...
155298,6284,0.000563,93436
155299,6285,0.000265,93436
155300,6286,0.000803,93436
155301,6287,0.000348,93436


In [6]:
df_master = wf.df_master

df_master

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_37,feature_38,feature_39,lookback_0,lookback_1,lookback_2,lookback_3,lookback_4,y,window
0,0.001527,0.001098,0.004587,0.001987,0.001109,0.001777,0.002703,0.000863,0.000652,0.000613,...,0.001261,0.002230,0.000822,0.001628,0.001087,0.001261,0.002230,0.000822,0.000719,"(0, 40)"
1,0.005466,0.005548,0.005260,0.004650,0.002337,0.002646,0.001095,0.001718,0.001386,0.001722,...,0.002704,0.005967,0.000982,0.000915,0.002798,0.002704,0.005967,0.000982,0.000963,"(0, 40)"
2,0.001633,0.000877,0.001704,0.001076,0.000759,0.000168,0.001103,0.000623,0.001896,0.001958,...,0.000522,0.000905,0.000241,0.000747,0.000840,0.000522,0.000905,0.000241,0.001694,"(0, 40)"
3,0.001556,0.000402,0.001299,0.000763,0.000339,0.000296,0.000407,0.000409,0.002330,0.000162,...,0.000600,0.000898,0.000488,0.000582,0.001683,0.000600,0.000898,0.000488,0.000395,"(0, 40)"
4,0.000487,0.000866,0.000314,0.000401,0.000703,0.000348,0.000169,0.000067,0.000035,0.000216,...,0.001239,0.002149,0.000766,0.001872,0.003063,0.001239,0.002149,0.000766,0.001984,"(0, 40)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111720,0.000223,0.000496,0.000130,0.000480,0.000210,0.000103,0.000165,0.000508,0.000134,0.000400,...,0.000193,0.000137,0.000192,0.000457,0.000071,0.000193,0.000137,0.000192,0.000171,"(6248, 6288)"
3111721,0.000183,0.000197,0.000115,0.000105,0.000312,0.001559,0.000444,0.000193,0.000454,0.000293,...,0.000039,0.000152,0.000164,0.000151,0.000066,0.000039,0.000152,0.000164,0.000178,"(6248, 6288)"
3111722,0.000732,0.000162,0.000599,0.000648,0.000355,0.000218,0.000518,0.000611,0.000377,0.000348,...,0.000225,0.000284,0.000576,0.000195,0.000117,0.000225,0.000284,0.000576,0.000123,"(6248, 6288)"
3111723,0.000694,0.000141,0.000340,0.002471,0.000172,0.000067,0.000420,0.000090,0.000416,0.000239,...,0.000045,0.000045,0.000133,0.000140,0.000155,0.000045,0.000045,0.000133,0.000047,"(6248, 6288)"
